In [10]:
import numpy 
import pandas as pd
from pandas import read_csv
import math
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.metrics import mean_squared_error
from keras.layers import BatchNormalization, Dropout
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
AI_Method = "/content/gdrive/My Drive/Colab Notebooks/Nal7/12188MT-1-ANN-SSA-Selected5b"
Var_LRs=[1e-2]# 1e-4,1e-5,1e-6,1e-7,1e-8,1e-9
Var_Decays=[1e-2] # 1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9
Var_epochs=[100]# 300,500]
 
Streamflow=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/12188MT-1SSA-1.csv', delimiter=',')

In [12]:
import numpy as np
x= Streamflow.drop('Q',axis=1)
Y= Streamflow['Q']
x= x.drop('Wind',axis=1)
 
x= x.drop('Q-1',axis=1)
x= x.drop('Q-2',axis=1)
x= x.drop('Q-3',axis=1)
x= x.drop('MTI',axis=1)
x= x.drop('Prep',axis=1)
x= x.drop('Humid',axis=1)
x= x.drop('Temp',axis=1)
x= x.drop('MeanQ',axis=1)
x= x.drop('Comp0',axis=1)
 
x.head()
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Cemp1   237 non-null    float64
 1   Comp2   237 non-null    float64
 2   Comp3   237 non-null    float64
dtypes: float64(3)
memory usage: 5.7 KB


In [13]:
X=np.array(x)
y=np.array(Y)
print (X.shape[1])

3


In [14]:
train_size_y = int(len(y) * 0.70)
train_size_X = int(len(X) * 0.70)
y_train, y_test = y[0:train_size_y], y[train_size_y:len(y)]
X_train, X_test = X[0:train_size_X], X[train_size_X:len(X)]

input=X_train[1:]
input_shape=X_train.shape[1:]

In [15]:
from datetime import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as geek

In [16]:
startTime = datetime.now()
fileOfSummary = open(AI_Method+'_Summary_'+str(datetime.now()).replace(":", ".")+'.csv', "w")
fileOfSummary.write("Method,LR,DE,Epoch,RunTime,CCTr,CCTt,NSTr,NSTt,RMSETr,RMSETt,MAETr,MAETt"+str(X.shape[1])+"\n")

74

In [17]:
for e_LR in Var_LRs:
    for e_decay in Var_Decays:
        for e_epoch in Var_epochs:
            
            def create_ANN():
                model = Sequential()
                model.add(Dense(units=100, input_shape=X_train.shape[1:]))
                model.add(BatchNormalization())
                model.add(Activation(activation='relu'))
                model.add(Dropout(0.2))
                model.add(BatchNormalization())
                model.add(Dropout(0.1))
                model.add(Dense(1, activation='relu'))
                return model
            model = create_ANN()
            
            optimizer = keras.optimizers.Adam(lr=e_LR, decay=e_decay)
            model.compile(optimizer=optimizer,loss='mean_squared_error')
        
            print(" ")
            print(" ")
            print("LRs:",e_LR)
            print("Decays:",e_decay)
            print("epochs:",e_epoch)
            history = model.fit(X_train, y_train, epochs=e_epoch, batch_size=256, verbose=0,  validation_data=(X_test, y_test), shuffle=True)
            
            hist_df = pd.DataFrame(history.history) 
            Time_elasped= datetime.now() - startTime
            print('\nTime elapsed: ', Time_elasped)
 
            Train = model.predict(X_train)
            Test = model.predict(X_test)
 
            FileName=AI_Method+'-LR'+str(e_LR)+'-DE'+str(e_decay)+'-'+str(e_epoch)
            np.savetxt(FileName+'_Train.csv', Train)
            np.savetxt(FileName+'_Test.csv', Test)
            #print('a',y_train.shape,Train.shape,len(y_train),len(Train))
            Train = geek.asarray(Train)# np.loadtxt(FileName+'_Train.csv')
            Test = geek.asarray(Test)#np.loadtxt(FileName+'_Test.csv')
 
            Train = Train.reshape(Train.shape[0])
            Test = Test.reshape(Test.shape[0])
 
            print(" ")
            print("LRs:",e_LR)
            print("Decays:",e_decay)
            print("epochs:",e_epoch)
            #print('b',y_train.shape,Train.shape,len(y_train),len(Train))
            print("Train ==>")
            CC_Train=np.corrcoef(y_train,Train)
            print("CC_Train = %.3f" %CC_Train[0,1])
 
            NSTr=1 - sum((Train-y_train)**2)/sum((y_train-np.mean(y_train))**2)
            print("NSTr = %.2f" %NSTr)
 
            rootMeanSquaredErrorTr = sqrt(mean_squared_error(y_train, Train))
            print("RMSE = %.2f" % rootMeanSquaredErrorTr)
            MAETr=mean_absolute_error(y_train, Train)
            print("MAE = %.2f" %  MAETr)
 
            print("Test ==>")
            CC_Test=np.corrcoef(y_test,Test)
            print("CC_Test = %.3f" %CC_Test[0,1])
 
            NSTt=1 - sum((Test-y_test)**2)/sum((y_test-np.mean(y_test))**2)
            print("NSTt = %.2f" %NSTt)
 
            rootMeanSquaredErrorTt = sqrt(mean_squared_error(y_test, Test))
            print("RMSE = %.2f" % rootMeanSquaredErrorTt)
            MAETt=mean_absolute_error(y_test, Test)
            print("MAE = %.2f" %  MAETt)
 
            fileOfSummary.write(AI_Method+','+str(e_LR)+','+str(e_decay)+','+str(e_epoch)+','+str(Time_elasped)+','+str(CC_Train[0,1])+','+str(CC_Test[0,1])+','+str(NSTr)+','+str(NSTt)+','+str(rootMeanSquaredErrorTr)+','+str(rootMeanSquaredErrorTt)+','+str(MAETr)+','+str(MAETt)+'\n')
            del model

 
 
LRs: 0.01
Decays: 0.01
epochs: 100

Time elapsed:  0:00:04.852606
 
LRs: 0.01
Decays: 0.01
epochs: 100
Train ==>
CC_Train = 0.742
NSTr = 0.29
RMSE = 1.99
MAE = 1.05
Test ==>
CC_Test = 0.720
NSTt = 0.22
RMSE = 1.52
MAE = 0.98


In [18]:
import io
buf = io.StringIO()
x.info(buf=buf)
s = buf.getvalue()

fileOfSummary.write(s)
fileOfSummary.close()

print("Finished")

Finished
